# Import de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil import tz

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = "/content/drive/MyDrive/Monografia/Dados Paranaguá/P2MOD/PARQUET"

data = {}

for file in os.listdir(path):
    if file.endswith('.parquet'):
        df = pd.read_parquet(os.path.join(path, file))
        data[file.split('.')[0]] = df

for key, df in data.items():
    print(f"{key}: {df.shape}")

ODAS_Corrente: (113094, 13)
Cattalini_Corrente: (273245, 7)
ODAS_Meteorologia: (104792, 4)
Cattalini_Meteorologia: (594571, 4)
Cattalini_Maregrafo: (273223, 2)
Porto_Maregrafo: (204582, 2)
Porto_Astronomica: (275851, 2)
Porto_Harmonico: (137926, 2)


# Estimativas de Frequências

In [3]:
# Código Dummy só para gerar algumas estatísticas
# Variar o i para determinado conjunto de dados 0 -> 5
# Variar o argumento de freq para frequência estimada por humano
# Ao checar as primeiras e últimas linhas dos dataframes

freq_estimadas = ["10min", "5min", "10min", "1min",
                  "5min", "5min", "5min", "10min"]

start_date = pd.to_datetime("2019-04-25 19:30:00+00:00")
end_date = pd.to_datetime("2021-12-08 15:00:00+00:00")

for i, (key, df) in enumerate(data.items()):
    print(f"Conjunto de Dados.....: {key}")
    print(f"Frequência Estimada...: {freq_estimadas[i]}")
    print(f"Número de Dados.......: {len(df)}")
    print(f"Número Estimado.......: {len(pd.date_range(start=start_date, end=end_date, freq=freq_estimadas[i]))}")
    print(f"Porcentagem de Falta..: {(1 - len(df) / len(pd.date_range(start=start_date, end=end_date, freq=freq_estimadas[i]))) * 100:.3f}%")
    print()


Conjunto de Dados.....: ODAS_Corrente
Frequência Estimada...: 10min
Número de Dados.......: 113094
Número Estimado.......: 137926
Porcentagem de Falta..: 18.004%

Conjunto de Dados.....: Cattalini_Corrente
Frequência Estimada...: 5min
Número de Dados.......: 273245
Número Estimado.......: 275851
Porcentagem de Falta..: 0.945%

Conjunto de Dados.....: ODAS_Meteorologia
Frequência Estimada...: 10min
Número de Dados.......: 104792
Número Estimado.......: 137926
Porcentagem de Falta..: 24.023%

Conjunto de Dados.....: Cattalini_Meteorologia
Frequência Estimada...: 1min
Número de Dados.......: 594571
Número Estimado.......: 1379251
Porcentagem de Falta..: 56.892%

Conjunto de Dados.....: Cattalini_Maregrafo
Frequência Estimada...: 5min
Número de Dados.......: 273223
Número Estimado.......: 275851
Porcentagem de Falta..: 0.953%

Conjunto de Dados.....: Porto_Maregrafo
Frequência Estimada...: 5min
Número de Dados.......: 204582
Número Estimado.......: 275851
Porcentagem de Falta..: 25.836%

C

# Regularização

In [4]:
data_grid = data.copy()

def to_grid_like(dataframe, freq="1min"):
    dataframe.set_index("Datetime", inplace=True)
    dataframe = dataframe.asfreq(freq=freq)
    dataframe.reset_index(inplace=True)
    return dataframe

In [5]:
data_grid['ODAS_Corrente'] = to_grid_like(data_grid['ODAS_Corrente'], freq="1min")
data_grid['Cattalini_Corrente'] = to_grid_like(data_grid['Cattalini_Corrente'], freq="1min")
data_grid['ODAS_Meteorologia'] = to_grid_like(data_grid['ODAS_Meteorologia'], freq="1min")
data_grid['Cattalini_Meteorologia'] = to_grid_like(data_grid['Cattalini_Meteorologia'], freq="1min")
data_grid["Cattalini_Maregrafo"] = to_grid_like(data_grid["Cattalini_Maregrafo"], freq="1min")
data_grid["Porto_Astronomica"] = to_grid_like(data_grid["Porto_Astronomica"], freq="1min")
data_grid["Porto_Harmonico"] = to_grid_like(data_grid["Porto_Harmonico"], freq="1min")

In [6]:
data_grid["Porto_Maregrafo"]["Datetime"] = data_grid["Porto_Maregrafo"]['Datetime'].dt.round('5min')
data_grid["Porto_Maregrafo"].drop_duplicates(subset=['Datetime'], inplace=True)
data_grid["Porto_Maregrafo"] = to_grid_like(data_grid["Porto_Maregrafo"], freq="1min")

In [7]:
import pandas as pd
import numpy as np

def regularize_start_end_date(dataframe, start_date, end_date):
    inf_comp_df = pd.DataFrame()  # DataFrame para preenchimento antes do intervalo
    sup_comp_df = pd.DataFrame()  # DataFrame para preenchimento após o intervalo

    # Adiciona dados antes do início do DataFrame, se necessário
    if dataframe["Datetime"].iloc[0] > start_date:
        inf_comp = pd.date_range(
            start=start_date,
            end=dataframe["Datetime"].iloc[0],
            freq='1min'
        )[:-1]  # Remove a última data para evitar sobreposição
        additions = len(inf_comp)
        inf_comp_df = {
            col: [np.nan] * additions for col in dataframe.columns
        }
        inf_comp_df['Datetime'] = inf_comp
        inf_comp_df = pd.DataFrame(inf_comp_df)

    # Adiciona dados após o final do DataFrame, se necessário
    if dataframe["Datetime"].iloc[-1] < end_date:
        sup_comp = pd.date_range(
            start=dataframe["Datetime"].iloc[-1],
            end=end_date,
            freq='1min'
        )[1:]  # Remove a primeira data para evitar sobreposição
        additions = len(sup_comp)
        sup_comp_df = {
            col: [np.nan] * additions for col in dataframe.columns
        }
        sup_comp_df['Datetime'] = sup_comp
        sup_comp_df = pd.DataFrame(sup_comp_df)

    # Combina os DataFrames: antes, original, e após
    return pd.concat([inf_comp_df, dataframe, sup_comp_df], ignore_index=True)

In [8]:
for key, df in data_grid.items():
    data_grid[key] = regularize_start_end_date(df, start_date, end_date)
    data_grid[key] = data_grid[key].set_index("Datetime")

dfs_adjusted = []

for key, df in data_grid.items():
    # Renomeia as colunas (exceto 'Datetime') para incluir a key
    df_renamed = df.rename(columns={col: f"{key}_{col}" for col in df.columns if col != "Datetime"})
    dfs_adjusted.append(df_renamed)

# Faz a junção com base na coluna 'Datetime'
merged_df = dfs_adjusted[0]
for df in dfs_adjusted[1:]:
    merged_df = merged_df.merge(df, on="Datetime", how="outer")

p2mod_gridlike = merged_df.copy()

In [9]:
export = "/content/drive/MyDrive/Monografia/Dados Paranaguá/P2MOD"
p2mod_gridlike.reset_index(inplace=True)
p2mod_gridlike.to_parquet(os.path.join(export, "P2MOD_GridLike.parquet"))